In [2]:
import json

from experiment_project.utils.initial.util import init_sys_env
from experiment_project.utils.files.read import read_yaml
from experiment_project.utils.knowledge_graph.conn import execute_cypher_statements
from experiment_project.utils.files.split import split_txt_by_langchain
import dspy
from experiment_project.utils.knowledge_graph.util import generate_cypher_statements
import os
from dspy.teleprompt import BootstrapFewShot
from dspy.retrieve.neo4j_rm import Neo4jRM
import time

In [3]:
init_sys_env(proxy_url='http://192.168.31.215:10890')
secret_env_file = '/mnt/c/Users/chenzi/Desktop/project/experiment_project/env_secret_config.yaml'
api_configs = read_yaml(secret_env_file)
model_config = api_configs.get('openai')
turbo = dspy.OpenAI(model=model_config.get('model'), max_tokens=3046,api_key=model_config.get('api_key'))
# turbo = dspy.OllamaLocal(model='qwen:32b-text-v1.5-q4_0',base_url='http://192.168.0.75:11434')
dspy.settings.configure(lm=turbo)

uri = 'bolt://localhost:7687'
user = 'cc'
password = 'Tt66668888..'
os.environ['NEO4J_URI'] = uri
os.environ['NEO4J_USERNAME'] = user
os.environ['NEO4J_PASSWORD'] = password
os.environ['OPENAI_API_KEY'] = model_config.get('api_key')


In [5]:
# retriever_model = Neo4jRM(
#     index_name="test_index_book",
#     text_node_property="text"
# )
# 
# results = retriever_model("汪淼是谁？", k=3)
# 
# for passage in results:
#     print("Document:", passage, "\n")

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `db.index.vector.queryNodes`: Caused by: java.lang.IllegalArgumentException: There is no such vector schema index: test_index_book}

In [4]:
from langchain.graphs import Neo4jGraph



graph = Neo4jGraph(
    url=uri, 
    username=user, 
    password=password
)

ValueError: Could not use APOC procedures. Please ensure the APOC plugin is installed in Neo4j and that 'apoc.meta.data()' is allowed in Neo4j configuration 

In [ ]:
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI

# Set up the GraphCypherQAChain
cypher_chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=ChatOpenAI(temperature=0, model=model_config.get('model')),
    qa_llm=ChatOpenAI(temperature=0, model=model_config.get('model')),
    validate_cypher=True,
    verbose=True
)

# Run the query
response = cypher_chain.run("汪淼是谁？")

